In [ ]:
from mesa import Agent, Model
from mesa.time import StagedActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid
from mesa.batchrunner import BatchRunner
from sklearn.neighbors import KernelDensity

import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import random
%run ./age_distribution.ipynb

In [ ]:
class BaseModel(Model):
    """A multistrain AT-MT model with fixed pop size.
    
    Inputs:
    
        agent_type: (class)Agent class
        n_pop: (int) Host population size
        n_infected_list: (list of int) Initial infected number of hosts with each strain
        avg_contacted_agents:(float) Average number of contacts between agents.
        prob_of_transmission_list: (list of float) Probability of transmission of each strain
                    in each contact
        strain_list: (a list of lists) List of strains where each strain is represented by a list with
                    [a,m] structure where a & m integers represent antigenic type and metabolic type of
                    strain, reppectively.
        avg_recovery_time: (float) Average infectious period of infected agents. This represents
                    mean value of Gamma distribution of infection duration.

        n_iterations: (int) Time horizon of the model

        agentDataTime: (int) must be set 0 when running the simulations. It is used to print population 
                    details for debugging.

        no_loci_AT: (int) Number of loci representing ATs.
        no_loci_MT: (int) Number of loci representing ATs.
        max_no_infections: (int) Max number of co-infections an agent can have 
        immunity_duration: (float) Average duration of immunity after infections. This represents
                    mean value of Gamma distribution of duration of immunity.            
        timeinterval_diversity: (int) The time interval in which period diversity is calculated.
        migration_rate: (float in [0,1]) The proportion of population who migrates at each time step
        carriage_percentage:(float in [0,1]) The proportion of emigrated agents who carry infections
        seed: (int) Random seed
        intervention_time: (int) Time of intervention where R0 value is halved.
        birthdeath: (boolean) represents whether there are birth-death processes in the model
        kd: Kernel density function of age distribution
        deathrates: Age-specific death rates

    """
    
    def __init__(self,agent_type,n_pop,n_infected_list, avg_contacted_agents, 
                 prob_of_transmission_list, 
                 strain_list, avg_recovery_time,n_iterations,
                 agentDataTime,no_loci_AT,
                no_loci_MT,max_no_infections,immunity_duration,
                 timeinterval_diversity,migration_rate, carriage_percentage,seed,intervention_time,
                 birthdeath=False, kd=kd,deathrates=deathrates):
        self.random = random.Random(seed)
        self.np_rng = np.random.default_rng(seed)
        self.agent_type = agent_type
        self.intervention_time = intervention_time
        #migration rate calculated as ~Poisson(migration_rate*N)
        #if migration rate ==0, then there will be no migration.
        self.migration_rate = migration_rate
        #infection carriage rate of migrated people is calculated by the carriage_percentage
        #if the percentage>=0, some migrated agents set as infected by a randomly
        #selected strain.
        self.carriage_percentage = carriage_percentage
        #birthdeath, a boolean represents whether 
        #there will be a bith death process or not
        self.birthdeath = birthdeath
        self.deathrates = deathrates
        
        #self age density function
        self.kd = kd
        
        #number of AT and MT loci
        self.no_loci_AT = no_loci_AT  #no of loci representing AT
        self.no_loci_MT = no_loci_MT #no of loci representing MT
        self.n_pop = n_pop   #population size
        
        #avg # of neighbors of each agent
        self.avg_contacted_agents = avg_contacted_agents 
        self.prob = self.avg_contacted_agents / self.n_pop
        self.next_id = 0 #id of next agent that is going to be created
        
        #max # of infection that each agent can carry at any time point
        self.max_no_infections=max_no_infections
        
        #constant immunity duration for all strain infections
        self.immunity_duration=immunity_duration
        #time interval (u value from Chisholm et al. 2020) to calculate period diversity
        self.u=timeinterval_diversity
        self.ulist=[]
        self.ulist_at=[]
        self.ulist_mt=[]
        #number of infections from each strain:
        self.nlist=[]
        #last introduction of strains to the population:
        self.last_introduction_of_strains=[]
        self.last_introduction_of_ats=[]
        self.no_reintroduction_of_strains=0
        self.no_reintroduction_of_ats=0
        self.total_no_introduction_of_strains=0
        self.total_no_introduction_of_ats=0
        #is the strain gone extinct in this time step:
        self.is_extinction_of_strains_current=[]
        self.is_extinction_of_ats_current=[]
        #duration of strains in community:
        self.duration_in_community_strain_dict = {}
        self.duration_in_community_at_dict = {}
        #number of incoming agents in each time step
        self.noEmmigratedAgents=0
        #reff dict of each strain
        self.reff_dict={}
        #reff dict of each at of strains
        self.reff_at_dict={}
        #reff pathogen dict
        self.reff_pathogen=[]
        #moving window reff dict of each strain
        self.moving_reff_dict={}
        #moving window reff dict of each at of strains
        self.moving_reff_at_dict={}
        #moving window reff pathogen dict
        self.moving_reff_pathogen={}
        self.reff_moving_interval=2*(avg_recovery_time)
        
        #G= model network: random network with avg edge number
        self.prob = self.avg_contacted_agents / self.n_pop
        self.G = nx.erdos_renyi_graph(n=self.n_pop, p=self.prob)  
        #create network G
        for node_id in self.G.nodes:
            self.G.nodes[node_id]['agent'] = None
        
        #create agent list and set the activation function
        self.schedule = StagedActivation(self,stage_list=["step1", "step2","advance"],shuffle=True)
        #List of prob of transmission of each strain
        self.prob_of_transmission_list=prob_of_transmission_list
        #avg recovery time=infection duration of infections
        self.avg_recovery_time=avg_recovery_time
        #total time step of the simulation
        self.n_iterations=n_iterations
        #list of initial # of infected agents from each strain
        self.n_infected_list=n_infected_list
        #strain list
        self.strain_list=strain_list
        #from strain list, create AT/MT lists:
        #aggreagated strain list based on their AT/MT values
        if self.no_loci_AT >= 0:
            self.slicing_AT_start = 0
            self.slicing_AT_end = self.no_loci_AT
            
            self.list_at = [self.strain_list[k][self.slicing_AT_start:self.slicing_AT_end] \
                          for k,l in enumerate(self.strain_list)]
            self.list_at.sort()
            self.unique_list_at = [list(x) for x in set(tuple(x) for x in self.list_at)]
            self.unique_list_at.sort()
            
        if self.no_loci_MT >= 0:
            self.slicing_MT_start = self.no_loci_AT
            self.slicing_MT_end = self.no_loci_AT + self.no_loci_MT
            self.list_mt = [self.strain_list[k][self.slicing_MT_start:self.slicing_MT_end] 
                          for k,l in enumerate(self.strain_list)]
            self.list_mt.sort()
        
        self.time=0
        
        #agentDataTime, f100 and df100 are used to keep track of some of agent values
        #these are only used for sanity check. And they are in function only
        #when agentDataTime set as greater than zero.
        self.agentDataTime=agentDataTime
        self.f100=True
        self.df100=pd.DataFrame()
        
        #data that is going to be collected in each time step.
        #see model_output_functions for more details.
        self.datacollector = DataCollector({"Infected": number_infected,
                                            "Susceptible": number_susceptible,
                                            "Immune": number_immune,
                                           "TotalPop": total_pop,
                                           "TotalInfected": total_number_infected,
                                           #"SimpsonsRecIndex":simpson_rec,
                                           # "SimpsonsRecIndexAT":simpson_rec_at,
                                            #"SimpsonsRecIndexMT":simpson_rec_mt,
                                           "DiversityIndex":diversity_index,
                                           #"HammingDistanceAT":hamming_distance_AT,
                                           #"HammingDistanceMT":hamming_distance_MT,
                                           "CirculatingNoStrains":circulating_no_strains,
                                            #"MeanCumInfections": mean_cum_infections,
                                            #"CirculatingNoMTs":circulating_no_MTs,
                                            "CirculatingNoATs":circulating_no_ATs,
                                            "PeriodDiversity": period_diversity,
                                            "PeriodDiversityAT": period_diversity_at,
                                            #"PeriodDiversityMT": period_diversity_mt,
                                            "Reff": calculate_reff,
                                            "ReffAT":calculate_reff_at,
                                            "ReffPathogen":calculate_reff_pathogen,
                                            "ReffWeekly": calculate_weekly_reff,
                                            "ReffATWeekly":calculate_weekly_reff_at,
                                            "ReffPathogenWeekly":calculate_weekly_reff_pathogen,
                                            "CumNoOfReintroductionStrains":cum_no_reintroduction_strains,
                                            "CumNoReintroductionAT":cum_no_reintroduction_ats,
                                            #"DurationInCommunityStrain":duration_in_community_strain,
                                            #"DurationInCommunityAT":duration_in_community_at,
                                            #"CumNoOfIntroductionStrains":cum_no_introduction_strains,
                                            #"CumNoIntroductionAT":cum_no_introduction_ats,
                                           })
        

            
        self.create_agents()
        self.infect_agents()
        self.assign_immunity()
        self.create_ulists()
        self.create_reff_dicts()
        self.running = True
        self.datacollector.collect(self)
        #run the model
        self.run_model(self.n_iterations)
        
    def create_agents(self):
        """create agents"""
        # Create agents
        #generate ages
        while True:
            #sample from density
            generated_ages=self.kd.sample(round(self.n_pop*1.5),random_state=self.random.randint(0,9999)) #,random_state=0)
            generated_ages=[round(x[0]) for x in generated_ages]
            myages=[x for x in generated_ages if (x > 0 and x<=100)  ]
            if len(myages) >= self.n_pop:
                break
        allages = self.random.sample(myages, self.n_pop)   
        
        
        for i, node in enumerate(self.G.nodes()):
            #assign age value to the initial agents
            age1=allages[i]
            #assign deathage value to the initial agents
            #create agents with no immunity and infectiond
            a = self.agent_type(i, self,self.agent_type,infection_list=[], immune_list=[],  
                      age=age1,
                      leftRecoveryTime=[]) 
            #add agent to the agent list        
            self.schedule.add(a)
            self.next_id=self.next_id+1
            #place agent in a node
            self.G.nodes[node]['agent'] = a
        
        #print age dist if it's asked
        if (self.agentDataTime != 0):
            agedist_agents=[a.age for a in self.schedule.agents]
            ax = plt.subplot(111)
            ax.hist(agedist_agents,alpha=0.5)
            plt.show()
            display(self.deathrates)
            
    def infect_agents(self):
        """infects some agents/nodes""" 
        for i,n in enumerate(self.n_infected_list):
            sus_nodes=[]
            for node in self.G.nodes:
                a=self.G.nodes[node]['agent']
                not_list=a.immune_list + a.infection_list
                
                nodes=sum([1 if (self.strain_list[i][0:] ==\
                                 not_list[j][self.slicing_AT_start:self.slicing_AT_end] or 
                                 self.strain_list[i][self.slicing_MT_start:self.slicing_MT_end] ==\
                                 not_list[j][self.slicing_MT_start:self.slicing_MT_end]) else 
                           0 for j in range(len(not_list))])
                if nodes == 0:
                    sus_nodes.append(node)
                
            infected_nodes = self.random.sample(sus_nodes, n)
            for node in infected_nodes:
                a = self.G.nodes[node]['agent']
                a.becomeInfected(self.strain_list[i])
    
    def assign_immunity(self):
        pass
        #assign immunity to some agents
        """
        for i,strain in enumerate(self.strain_list):
            num_immunes = round(self.n_pop * random.uniform(0.35, 0.60))
            immune_agents = random.sample(self.schedule.agents, num_immunes)
            
            for agent in immune_agents:
                agent.immune_list.append(strain)
                initial_immunity = round(float(np.random.gamma(shape=20, 
                                                scale=self.immunity_duration/20, size=1)))
                agent.leftImmunityTime.append(random.randrange(0, initial_immunity))
        """
            
    def update_ulists(self):
        """
        updates ulists which keeps track of period diversity
        #Chisholm (2020) period diversity
        
        -also updates duration_strains/ats_dicts:How long each strain stays in the community before an extinction.
        """
        
        self.n_list=[]
        for i,s in enumerate(self.strain_list):
            #no of infection from each strains
            no_current_infections=sum([1 if s in a.infection_list else 0 for a 
                               in self.schedule.agents])
            self.n_list.append(no_current_infections) 
            
            if (no_current_infections == 0 and self.is_extinction_of_strains_current[i] == 1):
                #new- update duration_strains_dicts:
                #the extinction is not current anymore:
                self.is_extinction_of_strains_current[i]=0
                duration_in_community_current=self.time - self.last_introduction_of_strains[i] +1
                self.duration_in_community_strain_dict["%s"%s].append(duration_in_community_current)
                
        if self.time == (self.n_iterations -1):    
            #at the end of the simulation update the duration in community for the strains 
            #by adding last duration without extinction
            for i, key in enumerate(self.duration_in_community_strain_dict.keys()):
                duration_in_community_current=self.n_iterations - self.last_introduction_of_strains[i]
                self.duration_in_community_strain_dict[key].append(duration_in_community_current)
            for i,key in enumerate(self.duration_in_community_at_dict.keys()):
                duration_in_community_current=self.n_iterations - self.last_introduction_of_ats[i]
                self.duration_in_community_at_dict[key].append(duration_in_community_current)
                        
                
        
        #update ulists:
        if self.time<= (self.u-1):
            for i in range(len(self.strain_list)):
                self.ulist[i].append(self.n_list[i])
            if self.no_loci_AT >0: #update ulist_at
                df = pd.DataFrame({'list_at':self.list_at, 'ulist':self.n_list})
                #display(df)
                gp = df.groupby(['list_at'])
                res = df.groupby(df['list_at'].map(tuple))['ulist'].sum()
                res2=[np.sum(res[x]) for x in range(len(res))]
                for j in range(len(res2)):
                    self.ulist_at[j].append(res2[j])
                    
                    if (res2[j] == 0 and self.is_extinction_of_ats_current[j] == 1):
                        #new- update duration_strains_dicts:
                        #the extinction is not current anymore:
                        self.is_extinction_of_ats_current[j]=0
                        duration_in_community_current=self.time - self.last_introduction_of_ats[j]+1
                        self.duration_in_community_at_dict["%s"%self.unique_list_at[j]].append(duration_in_community_current)
                
            if self.no_loci_MT >0: #update ulist_mt
                df = pd.DataFrame({'list_mt':self.list_mt, 'ulist':self.n_list})
                gp = df.groupby(['list_mt'])
                res1 = df.groupby(df['list_mt'].map(tuple))['ulist'].sum()
                res3=[np.sum(x) for x in res1]
                for j in range(len(res1)):
                    self.ulist_mt[j].append(res3[j])
        else: #time passes time interval u, so delete first element
            for i in range(len(self.strain_list)):
                self.ulist[i]=self.ulist[i][1:]
                self.ulist[i].append(self.n_list[i])
            if self.no_loci_AT >0: #update ulist_at
                df = pd.DataFrame({'list_at':self.list_at, 'ulist':self.n_list})
                gp = df.groupby(['list_at'])
                res = df.groupby(df['list_at'].map(tuple))['ulist'].sum()
                res2=[np.sum(x) for x in res]
                for j in range(len(res2)):
                    self.ulist_at[j]=self.ulist_at[j][1:]
                    self.ulist_at[j].append(res2[j])
                    
                    if (res2[j] == 0 and self.is_extinction_of_ats_current[j] == 1):
                        #new- update duration_strains_dicts:
                        #the extinction is not current anymore:
                        self.is_extinction_of_ats_current[j]=0
                        duration_in_community_current=self.time - self.last_introduction_of_ats[j]+1
                        self.duration_in_community_at_dict["%s"%s[self.slicing_AT_start:self.slicing_AT_end]].\
                        append(duration_in_community_current)
                
            if self.no_loci_MT >0: #update ulist_mt
                df = pd.DataFrame({'list_mt':self.list_mt, 'ulist':self.n_list})
                gp = df.groupby(['list_mt'])
                res1 = df.groupby(df['list_mt'].map(tuple))['ulist'].sum()
                res3=[np.sum(x) for x in res1]
                for j in range(len(res1)):
                    self.ulist_mt[j]=self.ulist_mt[j][1:]
                    self.ulist_mt[j].append(res3[j])
            
    def create_reff_dicts(self):
        """
        -creates reff counting dicts for all strains and antigenically aggregated strains.
        -also creates duration_in_community dicts for all strains and antigenically aggregated strains.
        
        
        -also creates last_introduction_of_strains/ats and is_extinction_of_strains/ats_current lists.
        """        
        self.moving_reff_pathogen["ReffPathogen"]=[]
        
        for s in self.strain_list:
            self.reff_dict["%s"%s]=[]
            self.moving_reff_dict["%s"%s]=[]
            self.duration_in_community_strain_dict["%s"%s]=[]
            self.last_introduction_of_strains.append(0)
            self.is_extinction_of_strains_current.append(1)
            
        
        if self.no_loci_AT >0:
            for s in self.unique_list_at:
                self.reff_at_dict["%s"%s]=[]
                self.moving_reff_at_dict["%s"%s]=[]
                self.duration_in_community_at_dict["%s"%s]=[]
                self.last_introduction_of_ats.append(0)
                self.is_extinction_of_ats_current.append(1)
        
    
    def clear_reff_dics(self):
        """ 
        -adds reff value to the moving reff dict where moving window is equal to the infection duration.
        -adds reff_dict to the moving reff dicts and
            clears reff dicts by keeping the key names at the end of each model step.
        -also clears duration_in_community_strain_dict and duration_in_community_at_dict 
            by keeping the key names at the end of each model step.
        """
                    
        if self.time < self.reff_moving_interval:
            for key,value in self.reff_dict.items():
                self.moving_reff_dict[key].append(value.copy())
                del value[:]
            if self.no_loci_AT >0:
                for key, value in self.reff_at_dict.items():
                    self.moving_reff_at_dict[key].append(value.copy())
                    del value[:]
            self.moving_reff_pathogen["ReffPathogen"].append(self.reff_pathogen.copy())        
            self.reff_pathogen=[]
        else:
            for key,value in self.reff_dict.items():
                self.moving_reff_dict[key]=self.moving_reff_dict["%s"%key][1:]
                self.moving_reff_dict[key].append(value.copy())
                del value[:]
            
            if self.no_loci_AT >0:
                for key, value in self.reff_at_dict.items():
                    self.moving_reff_at_dict["%s"%key]=self.moving_reff_at_dict["%s"%key][1:]
                    self.moving_reff_at_dict["%s"%key].append(value.copy())
                    del value[:]
                    
            self.moving_reff_pathogen["ReffPathogen"]=self.moving_reff_pathogen["ReffPathogen"][1:]
            self.moving_reff_pathogen["ReffPathogen"].append(self.reff_pathogen.copy())        
            
            self.reff_pathogen=[]
                 
    def create_ulists(self):
        """
        create ulists for period diversity.
        """
        
        self.n_list=[]
        for s in self.strain_list:
            #no of infection from each strains
            self.n_list.append(sum([1 if s in a.infection_list else 0 for a 
                               in self.schedule.agents])) 
        
        for s in self.strain_list:
            self.ulist.append([sum([1 if s in a.infection_list 
                                else 0 for a in self.schedule.agents])])
        if self.no_loci_AT >0:
            df = pd.DataFrame({'list_at':self.list_at, 'ulist':self.ulist})
            gp = df.groupby(['list_at'])
            res = df.groupby(df['list_at'].map(tuple))['ulist'].sum()
            res2=[np.sum(x) for x in res]
            self.ulist_at=list(map(lambda el:[el], res2))
        
        if self.no_loci_MT >0:
            df = pd.DataFrame({'list_mt':self.list_mt, 'ulist':self.ulist})
            gp = df.groupby(['list_mt'])
            res1 = df.groupby(df['list_mt'].map(tuple))['ulist'].sum()
            res3=[np.sum(x) for x in res1]
            self.ulist_mt=list(map(lambda el:[el], res3))
            
    def emigrateAgents(self,noEmmigratedAgents):
        """Adds noEmmigratedAgents number of new agents to the network."""
        
        #create list of new strains emigrated to the population
        new_strains_list=[]
        for k in range(noEmmigratedAgents):
            if self.random.random() < self.carriage_percentage:
                #the emigrated agent is infected
                #randomly assign a new strain
                new_strains_list.append(self.random.choice(self.strain_list))
            
        #check if new strains were already circulating in the population:
        unique_new_strain_list=[list(x) for x in set(tuple(x) for x in new_strains_list)]
        unique_at_list=[list(x) for x in set(tuple(x[self.slicing_AT_start:self.slicing_AT_end])\
                                             for x in unique_new_strain_list)]
        self.total_no_introduction_of_strains+=len(unique_new_strain_list)
        self.total_no_introduction_of_ats+=len(unique_at_list)
        for k in unique_new_strain_list:
            infections_new_strain=self.n_list[self.strain_list.index(k)]
            if infections_new_strain==0:
                #this is re-introduction of an extinct strain
                self.no_reintroduction_of_strains +=1
                #this strain hasn't gone extinction after last introduction (current intro)
                self.is_extinction_of_strains_current[self.strain_list.index(k)]=1
                #update last introduction time of strain
                self.last_introduction_of_strains[self.strain_list.index(k)]=self.time
                
                #check if that AT has gone extinction before:
                new_at=k[0:self.no_loci_AT]
                if self.no_loci_AT >0: #update ulist_at
                    df = pd.DataFrame({'list_at':self.list_at, 'ulist':self.n_list})
                    gp = df.groupby(['list_at'])
                    res = df.groupby(df['list_at'].map(tuple))['ulist'].sum()
                    res2=[np.sum(x) for x in res]
                    if res2[self.unique_list_at.index(new_at)]==0: 
                        #at has gone extintion before, it is now re-introduced
                        self.no_reintroduction_of_ats +=1
                        #this strain hasn't gone extinction after last introduction (current intro)
                        self.is_extinction_of_ats_current[self.unique_list_at.index(new_at)]=1
                        self.last_introduction_of_ats[self.unique_list_at.index(new_at)]=self.time
        
        if len(new_strains_list)<noEmmigratedAgents:
            new_strains_list=new_strains_list+[[]]*(noEmmigratedAgents-len(new_strains_list))
            
        for k in range(noEmmigratedAgents):    
            # and emmigrate another agent
            n_id=self.next_id
            #assign age and death age to emigrated agent
            age1=round(float(self.kd.sample(1,random_state=self.random.randint(0,9999))))
            #assign immune list to the emigrated agent
            random_id=self.random.choice([node_id for node_id in self.G.nodes])
            random_agent=self.G.nodes[random_id]['agent']
            immune_l=random_agent.immune_list
            a = self.agent_type(n_id,self,self.agent_type,infection_list=[], immune_list=immune_l,  
                         age=age1,#deathAge= deathAge1,
                         leftRecoveryTime=[]) 
            a.leftImmunityTime=random_agent.leftImmunityTime
            #infect the emigrated agent with carriage percentage
            if len(new_strains_list[k])!= 0:
                #infect the agent
                a.becomeInfected(new_strains_list[k])

            self.schedule.add(a) 
            #display("Migration: agent %s left and agent %s came"%(self.unique_id,n_id))    
            #create new node
            list_of_nodes = [node_id for node_id in self.G.nodes if 
                             self.random.random() < self.prob]
            #heterogeneous network
            self.G.add_node(n_id)
            self.G.add_edges_from((n_id,i) for i in list_of_nodes)
            #assign new agent 'a' to new node
            self.G.nodes[n_id]['agent'] = None
            self.G.nodes[n_id]['agent'] = a
            self.next_id=self.next_id+1
        self.noEmmigratedAgents=0
        
        
        
    def step(self):
        """Model step function: calls agent steps and collect data."""
        self.clear_reff_dics()
        self.schedule.step()
        self.update_ulists()
        self.emigrateAgents(self.noEmmigratedAgents)
        # collect data
        self.datacollector.collect(self)
        
    def run_model(self,n_iterations):
        """Model function that calls model step for # of iterations.
        -If there is an intervention at that point, R0 is halved.
        -If agentDataTime is not zero, it also prints the selected agent values of
            all the agents for 2 adjacent time steps:
            t=agentDataTime and t+1=agentDataTime+1."""
        for i in range(n_iterations):
            self.time=i
            self.step()
            if (self.time == self.intervention_time):
                display("Time: %s, Number of edges %s"%(self.time,len(self.G.edges())))
                to_remove=self.random.sample(self.G.edges(),k=len(self.G.edges())//2)
                self.G.remove_edges_from(to_remove)
                self.prob = self.prob / 2
                display("Number of edges %s"%len(self.G.edges()))
                
            if (self.time == n_iterations-1):
                if (self.agentDataTime != 0):
                    agedist_agents=[a.age for a in self.schedule.agents]
                    ax = plt.subplot(111)
                    ax.hist(agedist_agents,alpha=0.5, color="red")
                    plt.show()
                    display(self.df100)    
        
